# Federated Learning on MNIST using a CNN

### Imports and model specifications

First we make the official imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

And than those specific to PySyft. In particular we define remote workers `alice` and `bob`.

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

We define the setting of the learning task

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Data loading and sending to workers
We first load the data and transform the training Dataset into a Federated Dataset split across the workers using the `.federate` method. This federated dataset is now given to a Federated DataLoader. The test dataset remains unchanged.

In [5]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 16648958.51it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 444041.88it/s]
  1%|          | 16384/1648877 [00:00<00:11, 146612.81it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 6450865.81it/s]                           
8192it [00:00, 181632.26it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


### CNN specification
Here we use exactly the same CNN as in the official example.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define the train and test functions
For the train function, because the data batches are distributed across `alice` and `bob`, you need to send the model to the right location for each batch. Then, you perform all the operations remotely with the same syntax like you're doing local PyTorch. When you're done, you get back the model updated and the loss to look for improvement.

In [7]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

The test function does not change!

In [8]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Launch the training !

In [9]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896600
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440339
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.866960
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654266
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593402
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.455609
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.370832
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.303737
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.314273
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.369014
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.238090
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.187788
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.523821
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.225225
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.143864
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.268484
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.186659
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.305151
Train Epoch:

Train Epoch: 6 [1920/60032 (3%)]	Loss: 0.103697
Train Epoch: 6 [3840/60032 (6%)]	Loss: 0.011604
Train Epoch: 6 [5760/60032 (10%)]	Loss: 0.090397
Train Epoch: 6 [7680/60032 (13%)]	Loss: 0.112624
Train Epoch: 6 [9600/60032 (16%)]	Loss: 0.014472
Train Epoch: 6 [11520/60032 (19%)]	Loss: 0.087826
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.037394
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.098747
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.052957
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.043855
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.073001
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.047514
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.007711
Train Epoch: 6 [26880/60032 (45%)]	Loss: 0.019910
Train Epoch: 6 [28800/60032 (48%)]	Loss: 0.034876
Train Epoch: 6 [30720/60032 (51%)]	Loss: 0.084788
Train Epoch: 6 [32640/60032 (54%)]	Loss: 0.010765
Train Epoch: 6 [34560/60032 (58%)]	Loss: 0.026387
Train Epoch: 6 [36480/60032 (61%)]	Loss: 0.048259
Train Epoch: 6 [38400/60032 (64%)]	Loss: 0.006613
Train E

Et voilà! Here you are, you have trained a model on remote data using Federated Learning!